# Day 4: Production Patterns - Managing Hotel Reservations at Scale

**Week 18 - Enterprise GPU Management**

> *"A 5-star hotel doesn't just have rooms—it has health monitoring, failover procedures, resource management, and a staff trained for any situation."*

---

## 🎯 Learning Objectives

By the end of this lesson, you will:
1. **Implement** GPU health monitoring systems
2. **Design** failover mechanisms for GPU failures
3. **Apply** RAII patterns for automatic resource cleanup
4. **Build** production-ready GPU management frameworks

---

## 🏨 Concept Card: Managing Hotel Reservations

**Production MIG is like running a professional hotel chain:**

```
Hotel Management Dashboard:
┌─────────────────────────────────────────────────────────────┐
│           🏨 GPU Hotel Chain Management System              │
├─────────────────────────────────────────────────────────────┤
│  📊 Health Monitor (NVML)          🔄 Failover System       │
│  ┌─────────────────────────┐       ┌──────────────────────┐ │
│  │ GPU 0: 🟢 45°C 4.2GB/40  │       │ Primary: GPU 0       │ │
│  │ GPU 1: 🟢 48°C 8.1GB/40  │       │ Backup:  GPU 1       │ │
│  │ GPU 2: 🟡 72°C 38GB/40   │       │ Status: Ready        │ │
│  │ GPU 3: 🔴 OFFLINE        │       │ Last switch: Never   │ │
│  └─────────────────────────┘       └──────────────────────┘ │
├─────────────────────────────────────────────────────────────┤
│  📋 Resource Manager                🎫 Reservation System   │
│  ┌─────────────────────────┐       ┌──────────────────────┐ │
│  │ Contexts: 12 active     │       │ Queue: 5 pending     │ │
│  │ Streams: 48 created     │       │ Active: 12 workloads │ │
│  │ Memory: 156GB allocated │       │ SLA: 99.9% uptime    │ │
│  │ Auto-cleanup: ENABLED   │       │ Priority: ML > Viz   │ │
│  └─────────────────────────┘       └──────────────────────┘ │
└─────────────────────────────────────────────────────────────┘
```

**The Hotel Management Mapping:**
| Hotel Concept | Production GPU | Implementation |
|---------------|----------------|----------------|
| 🌡️ Room sensors | Health monitoring | NVML temperature/power/ECC |
| 🔄 Backup generator | GPU failover | Multi-GPU redundancy |
| 🧹 Housekeeping | RAII cleanup | Automatic resource release |
| 📝 Guest registry | Logging system | Timestamps, metrics, errors |
| 🏃 Fire drills | Error path testing | Verify recovery works |
| 📊 Occupancy tracking | Resource monitoring | Memory/SM utilization |

---

## Pattern 1: GPU Health Checker

In [ ]:
%%writefile gpu_health_check.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <nvml.h>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA error: %s\n", cudaGetErrorString(err)); \
            return false; \
        } \
    } while(0)

struct GPUHealth {
    int deviceId;
    bool cudaAccessible;
    bool memoryOk;
    bool computeOk;
    bool temperatureOk;
    bool eccOk;
    
    // Metrics
    size_t freeMemory;
    size_t totalMemory;
    unsigned int temperature;
    unsigned int powerUsage;
    unsigned int gpuUtilization;
};

__global__ void healthCheckKernel(int* result) {
    // Simple compute test
    int val = threadIdx.x + blockIdx.x * blockDim.x;
    val = val * 2 + 1;
    if (threadIdx.x == 0) {
        *result = val;
    }
}

bool checkGPUHealth(int deviceId, GPUHealth& health) {
    health.deviceId = deviceId;
    health.cudaAccessible = false;
    health.memoryOk = false;
    health.computeOk = false;
    health.temperatureOk = false;
    health.eccOk = true;  // Assume OK unless detected otherwise
    
    // Test CUDA accessibility
    CHECK_CUDA(cudaSetDevice(deviceId));
    health.cudaAccessible = true;
    
    // Memory check
    CHECK_CUDA(cudaMemGetInfo(&health.freeMemory, &health.totalMemory));
    health.memoryOk = (health.freeMemory > 0.1 * health.totalMemory);  // >10% free
    
    // Compute check
    int* d_result;
    int h_result = -1;
    CHECK_CUDA(cudaMalloc(&d_result, sizeof(int)));
    
    healthCheckKernel<<<1, 32>>>(d_result);
    CHECK_CUDA(cudaDeviceSynchronize());
    CHECK_CUDA(cudaMemcpy(&h_result, d_result, sizeof(int), cudaMemcpyDeviceToHost));
    CHECK_CUDA(cudaFree(d_result));
    
    health.computeOk = (h_result == 1);  // Expected: 0 * 2 + 1 = 1
    
    // NVML checks
    nvmlDevice_t nvmlDevice;
    if (nvmlDeviceGetHandleByIndex(deviceId, &nvmlDevice) == NVML_SUCCESS) {
        // Temperature
        if (nvmlDeviceGetTemperature(nvmlDevice, NVML_TEMPERATURE_GPU, 
                                      &health.temperature) == NVML_SUCCESS) {
            health.temperatureOk = (health.temperature < 85);  // <85°C
        }
        
        // Power
        nvmlDeviceGetPowerUsage(nvmlDevice, &health.powerUsage);
        health.powerUsage /= 1000;  // Convert to watts
        
        // Utilization
        nvmlUtilization_t util;
        if (nvmlDeviceGetUtilizationRates(nvmlDevice, &util) == NVML_SUCCESS) {
            health.gpuUtilization = util.gpu;
        }
        
        // ECC errors
        unsigned long long eccErrors;
        if (nvmlDeviceGetTotalEccErrors(nvmlDevice, NVML_MEMORY_ERROR_TYPE_UNCORRECTED,
                                         NVML_VOLATILE_ECC, &eccErrors) == NVML_SUCCESS) {
            health.eccOk = (eccErrors == 0);
        }
    }
    
    return health.cudaAccessible && health.memoryOk && 
           health.computeOk && health.temperatureOk && health.eccOk;
}

void printHealth(const GPUHealth& h) {
    printf("GPU %d Health:\n", h.deviceId);
    printf("  CUDA Accessible: %s\n", h.cudaAccessible ? "OK" : "FAIL");
    printf("  Memory: %s (%.1f/%.1f GB free)\n", 
           h.memoryOk ? "OK" : "LOW",
           h.freeMemory / (1024.0*1024.0*1024.0),
           h.totalMemory / (1024.0*1024.0*1024.0));
    printf("  Compute: %s\n", h.computeOk ? "OK" : "FAIL");
    printf("  Temperature: %s (%u°C)\n", 
           h.temperatureOk ? "OK" : "HIGH", h.temperature);
    printf("  ECC: %s\n", h.eccOk ? "OK" : "ERRORS DETECTED");
    printf("  Power: %u W, Utilization: %u%%\n", 
           h.powerUsage, h.gpuUtilization);
}

int main() {
    printf("=== GPU Health Check ===\n\n");
    
    nvmlInit();
    
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    
    int healthyGPUs = 0;
    for (int i = 0; i < deviceCount; i++) {
        GPUHealth health;
        bool isHealthy = checkGPUHealth(i, health);
        
        printHealth(health);
        printf("  Overall: %s\n\n", isHealthy ? "HEALTHY" : "UNHEALTHY");
        
        if (isHealthy) healthyGPUs++;
    }
    
    printf("=== Summary: %d/%d GPUs healthy ===\n", healthyGPUs, deviceCount);
    
    nvmlShutdown();
    return (healthyGPUs == deviceCount) ? 0 : 1;
}

In [ ]:
!nvcc -O3 -arch=sm_80 gpu_health_check.cu -o gpu_health_check -lnvidia-ml && ./gpu_health_check

## Pattern 2: Graceful GPU Failover

In [ ]:
%%writefile gpu_failover.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <vector>

class GPUManager {
public:
    struct GPUInfo {
        int id;
        bool available;
        size_t memory;
        int smCount;
    };
    
private:
    std::vector<GPUInfo> gpus_;
    int currentDevice_ = -1;
    
public:
    bool initialize() {
        int count;
        if (cudaGetDeviceCount(&count) != cudaSuccess) return false;
        
        for (int i = 0; i < count; i++) {
            GPUInfo info;
            info.id = i;
            info.available = true;
            
            cudaDeviceProp prop;
            if (cudaGetDeviceProperties(&prop, i) == cudaSuccess) {
                info.memory = prop.totalGlobalMem;
                info.smCount = prop.multiProcessorCount;
            }
            
            gpus_.push_back(info);
        }
        
        return !gpus_.empty();
    }
    
    int selectBestGPU() {
        int best = -1;
        size_t bestMemory = 0;
        
        for (auto& gpu : gpus_) {
            if (!gpu.available) continue;
            
            // Check actual free memory
            cudaSetDevice(gpu.id);
            size_t free, total;
            if (cudaMemGetInfo(&free, &total) == cudaSuccess) {
                if (free > bestMemory) {
                    bestMemory = free;
                    best = gpu.id;
                }
            } else {
                gpu.available = false;  // Mark as unavailable
            }
        }
        
        return best;
    }
    
    bool setDevice(int deviceId) {
        if (cudaSetDevice(deviceId) != cudaSuccess) {
            markUnavailable(deviceId);
            return false;
        }
        currentDevice_ = deviceId;
        return true;
    }
    
    void markUnavailable(int deviceId) {
        for (auto& gpu : gpus_) {
            if (gpu.id == deviceId) {
                gpu.available = false;
                printf("[FAILOVER] GPU %d marked unavailable\n", deviceId);
                break;
            }
        }
    }
    
    int failover() {
        printf("[FAILOVER] Attempting failover from GPU %d\n", currentDevice_);
        
        // Mark current as unavailable
        if (currentDevice_ >= 0) {
            markUnavailable(currentDevice_);
        }
        
        // Reset device to clear any errors
        cudaDeviceReset();
        
        // Find next best GPU
        int next = selectBestGPU();
        if (next >= 0) {
            if (setDevice(next)) {
                printf("[FAILOVER] Switched to GPU %d\n", next);
                return next;
            }
        }
        
        printf("[FAILOVER] No available GPUs!\n");
        return -1;
    }
    
    void printStatus() {
        printf("\n=== GPU Status ===\n");
        for (const auto& gpu : gpus_) {
            printf("GPU %d: %s (%.1f GB, %d SMs)\n",
                   gpu.id, 
                   gpu.available ? "Available" : "Unavailable",
                   gpu.memory / (1024.0*1024.0*1024.0),
                   gpu.smCount);
        }
        printf("Current: GPU %d\n\n", currentDevice_);
    }
};

__global__ void workloadKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = sqrtf((float)idx);
    }
}

bool executeWithFailover(GPUManager& mgr, float* d_data, int n) {
    const int maxRetries = 3;
    
    for (int retry = 0; retry < maxRetries; retry++) {
        int blockSize = 256;
        int numBlocks = (n + blockSize - 1) / blockSize;
        
        workloadKernel<<<numBlocks, blockSize>>>(d_data, n);
        
        cudaError_t err = cudaDeviceSynchronize();
        if (err == cudaSuccess) {
            return true;  // Success!
        }
        
        printf("[ERROR] Kernel failed: %s\n", cudaGetErrorString(err));
        
        // Attempt failover
        int newDevice = mgr.failover();
        if (newDevice < 0) {
            return false;  // No GPUs left
        }
        
        // Reallocate on new device
        cudaMalloc(&d_data, n * sizeof(float));
        printf("[RECOVERY] Retrying on GPU %d (attempt %d/%d)\n", 
               newDevice, retry + 2, maxRetries);
    }
    
    return false;
}

int main() {
    printf("=== GPU Failover Demo ===\n\n");
    
    GPUManager mgr;
    if (!mgr.initialize()) {
        printf("No GPUs found!\n");
        return 1;
    }
    
    mgr.printStatus();
    
    // Select best GPU
    int device = mgr.selectBestGPU();
    if (device < 0) {
        printf("No available GPUs!\n");
        return 1;
    }
    
    mgr.setDevice(device);
    printf("Selected GPU %d\n\n", device);
    
    // Allocate and run
    const int N = 1024 * 1024;
    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    
    printf("Executing workload...\n");
    if (executeWithFailover(mgr, d_data, N)) {
        printf("Workload completed successfully!\n");
    } else {
        printf("Workload failed after all retries!\n");
    }
    
    mgr.printStatus();
    cudaFree(d_data);
    
    return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_80 -std=c++14 gpu_failover.cu -o gpu_failover && ./gpu_failover

## Pattern 3: Resource RAII Wrappers

In [ ]:
%%writefile cuda_raii.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <memory>
#include <stdexcept>

// RAII wrapper for device memory
template<typename T>
class DeviceBuffer {
private:
    T* ptr_ = nullptr;
    size_t size_ = 0;
    
public:
    DeviceBuffer() = default;
    
    explicit DeviceBuffer(size_t count) : size_(count) {
        if (cudaMalloc(&ptr_, count * sizeof(T)) != cudaSuccess) {
            throw std::runtime_error("cudaMalloc failed");
        }
    }
    
    ~DeviceBuffer() {
        if (ptr_) cudaFree(ptr_);
    }
    
    // Move only
    DeviceBuffer(DeviceBuffer&& other) : ptr_(other.ptr_), size_(other.size_) {
        other.ptr_ = nullptr;
        other.size_ = 0;
    }
    
    DeviceBuffer& operator=(DeviceBuffer&& other) {
        if (this != &other) {
            if (ptr_) cudaFree(ptr_);
            ptr_ = other.ptr_;
            size_ = other.size_;
            other.ptr_ = nullptr;
            other.size_ = 0;
        }
        return *this;
    }
    
    // No copy
    DeviceBuffer(const DeviceBuffer&) = delete;
    DeviceBuffer& operator=(const DeviceBuffer&) = delete;
    
    T* get() { return ptr_; }
    const T* get() const { return ptr_; }
    size_t size() const { return size_; }
    size_t bytes() const { return size_ * sizeof(T); }
    
    void copyFrom(const T* host) {
        cudaMemcpy(ptr_, host, bytes(), cudaMemcpyHostToDevice);
    }
    
    void copyTo(T* host) const {
        cudaMemcpy(host, ptr_, bytes(), cudaMemcpyDeviceToHost);
    }
};

// RAII wrapper for streams
class CudaStream {
private:
    cudaStream_t stream_ = nullptr;
    
public:
    CudaStream() {
        cudaStreamCreate(&stream_);
    }
    
    explicit CudaStream(unsigned int flags) {
        cudaStreamCreateWithFlags(&stream_, flags);
    }
    
    ~CudaStream() {
        if (stream_) cudaStreamDestroy(stream_);
    }
    
    CudaStream(CudaStream&& other) : stream_(other.stream_) {
        other.stream_ = nullptr;
    }
    
    cudaStream_t get() { return stream_; }
    operator cudaStream_t() { return stream_; }
    
    void synchronize() { cudaStreamSynchronize(stream_); }
    bool query() { return cudaStreamQuery(stream_) == cudaSuccess; }
};

// RAII wrapper for events
class CudaEvent {
private:
    cudaEvent_t event_ = nullptr;
    
public:
    CudaEvent() {
        cudaEventCreate(&event_);
    }
    
    explicit CudaEvent(unsigned int flags) {
        cudaEventCreateWithFlags(&event_, flags);
    }
    
    ~CudaEvent() {
        if (event_) cudaEventDestroy(event_);
    }
    
    cudaEvent_t get() { return event_; }
    operator cudaEvent_t() { return event_; }
    
    void record(cudaStream_t stream = 0) {
        cudaEventRecord(event_, stream);
    }
    
    void synchronize() { cudaEventSynchronize(event_); }
    
    float elapsedMs(const CudaEvent& start) const {
        float ms;
        cudaEventElapsedTime(&ms, start.event_, event_);
        return ms;
    }
};

// Demo kernel
__global__ void addKernel(float* c, const float* a, const float* b, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) c[idx] = a[idx] + b[idx];
}

int main() {
    printf("=== CUDA RAII Wrappers Demo ===\n\n");
    
    const int N = 1024 * 1024;
    
    try {
        // Automatic cleanup on scope exit!
        DeviceBuffer<float> d_a(N);
        DeviceBuffer<float> d_b(N);
        DeviceBuffer<float> d_c(N);
        
        printf("Allocated 3 buffers: %.2f MB each\n", 
               d_a.bytes() / (1024.0*1024.0));
        
        // Prepare host data
        std::unique_ptr<float[]> h_a(new float[N]);
        std::unique_ptr<float[]> h_b(new float[N]);
        std::unique_ptr<float[]> h_c(new float[N]);
        
        for (int i = 0; i < N; i++) {
            h_a[i] = 1.0f;
            h_b[i] = 2.0f;
        }
        
        d_a.copyFrom(h_a.get());
        d_b.copyFrom(h_b.get());
        
        // Timing with RAII events
        CudaEvent start, stop;
        CudaStream stream;
        
        start.record(stream);
        
        int blockSize = 256;
        int numBlocks = (N + blockSize - 1) / blockSize;
        addKernel<<<numBlocks, blockSize, 0, stream>>>(d_c.get(), d_a.get(), d_b.get(), N);
        
        stop.record(stream);
        stream.synchronize();
        
        printf("Kernel time: %.3f ms\n", stop.elapsedMs(start));
        
        // Verify
        d_c.copyTo(h_c.get());
        bool correct = (h_c[0] == 3.0f && h_c[N-1] == 3.0f);
        printf("Result: %s\n", correct ? "CORRECT" : "INCORRECT");
        
    } catch (const std::exception& e) {
        printf("Error: %s\n", e.what());
        return 1;
    }
    
    // All resources automatically freed here!
    printf("\nResources automatically cleaned up.\n");
    
    return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_80 -std=c++14 cuda_raii.cu -o cuda_raii && ./cuda_raii

## Production Deployment Checklist

### Pre-Deployment
- [ ] All error checks in place
- [ ] Memory leak testing complete
- [ ] Stress testing passed
- [ ] Logging configured
- [ ] Metrics collection ready

### Runtime
- [ ] GPU health monitoring
- [ ] Memory usage alerts
- [ ] Temperature thresholds
- [ ] ECC error tracking
- [ ] Failover procedures

### Recovery
- [ ] Sticky error handling
- [ ] Device reset procedures
- [ ] State checkpointing
- [ ] Graceful degradation

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Complete these exercises to practice production deployment patterns:

1. **Exercise 1: GPU Health Monitor** - Build a comprehensive GPU health monitoring system
2. **Exercise 2: RAII Resource Manager** - Implement automatic resource cleanup with RAII
3. **Exercise 3: Failover Handler** - Design graceful GPU failure detection and recovery

In [ ]:
%%writefile production_patterns_exercises.cu
#include <cuda_runtime.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <string.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t error = call; \
        if (error != cudaSuccess) { \
            fprintf(stderr, "CUDA Error: %s at %s:%d\n", \
                    cudaGetErrorString(error), __FILE__, __LINE__); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

// =============================================================================
// Exercise 1: GPU Health Monitor
// Build a comprehensive GPU health monitoring system
// =============================================================================

struct GPUHealthMetrics {
    int deviceId;
    char name[256];
    size_t totalMemory;
    size_t freeMemory;
    size_t usedMemory;
    float memoryUsagePercent;
    int smCount;
    int temperature;  // Would need NVML for real temp
    bool isHealthy;
    char healthStatus[64];
};

class GPUHealthMonitor {
private:
    int deviceCount;
    GPUHealthMetrics* metrics;
    
    void updateMetrics(int deviceId) {
        CUDA_CHECK(cudaSetDevice(deviceId));
        
        cudaDeviceProp prop;
        CUDA_CHECK(cudaGetDeviceProperties(&prop, deviceId));
        
        metrics[deviceId].deviceId = deviceId;
        strncpy(metrics[deviceId].name, prop.name, 255);
        metrics[deviceId].smCount = prop.multiProcessorCount;
        
        CUDA_CHECK(cudaMemGetInfo(&metrics[deviceId].freeMemory, 
                                   &metrics[deviceId].totalMemory));
        
        metrics[deviceId].usedMemory = metrics[deviceId].totalMemory - 
                                       metrics[deviceId].freeMemory;
        metrics[deviceId].memoryUsagePercent = 
            100.0f * metrics[deviceId].usedMemory / metrics[deviceId].totalMemory;
        
        // Simulated temperature (would use NVML in production)
        metrics[deviceId].temperature = 45 + (rand() % 20);
        
        // Determine health status
        if (metrics[deviceId].memoryUsagePercent > 95.0f) {
            metrics[deviceId].isHealthy = false;
            strcpy(metrics[deviceId].healthStatus, "CRITICAL: Memory exhausted");
        } else if (metrics[deviceId].memoryUsagePercent > 85.0f) {
            metrics[deviceId].isHealthy = true;
            strcpy(metrics[deviceId].healthStatus, "WARNING: High memory usage");
        } else if (metrics[deviceId].temperature > 80) {
            metrics[deviceId].isHealthy = false;
            strcpy(metrics[deviceId].healthStatus, "CRITICAL: High temperature");
        } else {
            metrics[deviceId].isHealthy = true;
            strcpy(metrics[deviceId].healthStatus, "HEALTHY");
        }
    }

public:
    GPUHealthMonitor() {
        CUDA_CHECK(cudaGetDeviceCount(&deviceCount));
        metrics = new GPUHealthMetrics[deviceCount];
        srand(time(NULL));
    }
    
    ~GPUHealthMonitor() {
        delete[] metrics;
    }
    
    void collectMetrics() {
        for (int i = 0; i < deviceCount; i++) {
            updateMetrics(i);
        }
    }
    
    void printHealthReport() {
        printf("╔═══════════════════════════════════════════════════════════════════╗\n");
        printf("║                      GPU HEALTH REPORT                            ║\n");
        printf("╠═══════════════════════════════════════════════════════════════════╣\n");
        
        time_t now = time(NULL);
        printf("║ Timestamp: %-56s ║\n", ctime(&now));
        
        for (int i = 0; i < deviceCount; i++) {
            GPUHealthMetrics& m = metrics[i];
            
            printf("╠═══════════════════════════════════════════════════════════════════╣\n");
            printf("║ GPU %d: %-59s ║\n", i, m.name);
            printf("║   Memory: %.2f / %.2f GB (%.1f%% used)                            \n",
                   m.usedMemory / (1024.0 * 1024.0 * 1024.0),
                   m.totalMemory / (1024.0 * 1024.0 * 1024.0),
                   m.memoryUsagePercent);
            printf("║   SMs: %-3d  Temperature: %d°C (simulated)                        \n",
                   m.smCount, m.temperature);
            printf("║   Status: %-57s ║\n", m.healthStatus);
        }
        printf("╚═══════════════════════════════════════════════════════════════════╝\n");
    }
    
    bool allDevicesHealthy() {
        for (int i = 0; i < deviceCount; i++) {
            if (!metrics[i].isHealthy) return false;
        }
        return true;
    }
    
    int getHealthyDeviceCount() {
        int count = 0;
        for (int i = 0; i < deviceCount; i++) {
            if (metrics[i].isHealthy) count++;
        }
        return count;
    }
};

void exercise1_health_monitor() {
    printf("=== Exercise 1: GPU Health Monitor ===\n\n");
    
    GPUHealthMonitor monitor;
    
    // Collect and display metrics
    monitor.collectMetrics();
    monitor.printHealthReport();
    
    printf("\nHealth Summary:\n");
    printf("  All devices healthy: %s\n", monitor.allDevicesHealthy() ? "YES" : "NO");
    printf("  Healthy device count: %d\n\n", monitor.getHealthyDeviceCount());
}

// =============================================================================
// Exercise 2: RAII Resource Manager
// Implement automatic resource cleanup with RAII
// =============================================================================

template<typename T>
class CudaBuffer {
private:
    T* ptr;
    size_t count;
    bool ownsMemory;
    
public:
    CudaBuffer() : ptr(nullptr), count(0), ownsMemory(false) {}
    
    explicit CudaBuffer(size_t n) : count(n), ownsMemory(true) {
        cudaError_t err = cudaMalloc(&ptr, n * sizeof(T));
        if (err != cudaSuccess) {
            ptr = nullptr;
            count = 0;
            ownsMemory = false;
            fprintf(stderr, "CudaBuffer allocation failed: %s\n", 
                    cudaGetErrorString(err));
        } else {
            printf("  [RAII] Allocated %zu bytes at %p\n", n * sizeof(T), ptr);
        }
    }
    
    ~CudaBuffer() {
        if (ownsMemory && ptr != nullptr) {
            printf("  [RAII] Freeing buffer at %p\n", ptr);
            cudaFree(ptr);
        }
    }
    
    // Disable copy
    CudaBuffer(const CudaBuffer&) = delete;
    CudaBuffer& operator=(const CudaBuffer&) = delete;
    
    // Enable move
    CudaBuffer(CudaBuffer&& other) noexcept 
        : ptr(other.ptr), count(other.count), ownsMemory(other.ownsMemory) {
        other.ptr = nullptr;
        other.count = 0;
        other.ownsMemory = false;
    }
    
    CudaBuffer& operator=(CudaBuffer&& other) noexcept {
        if (this != &other) {
            if (ownsMemory && ptr) cudaFree(ptr);
            ptr = other.ptr;
            count = other.count;
            ownsMemory = other.ownsMemory;
            other.ptr = nullptr;
            other.count = 0;
            other.ownsMemory = false;
        }
        return *this;
    }
    
    T* get() { return ptr; }
    const T* get() const { return ptr; }
    size_t size() const { return count; }
    bool valid() const { return ptr != nullptr; }
    
    cudaError_t copyFromHost(const T* hostData, size_t n) {
        if (!valid() || n > count) return cudaErrorInvalidValue;
        return cudaMemcpy(ptr, hostData, n * sizeof(T), cudaMemcpyHostToDevice);
    }
    
    cudaError_t copyToHost(T* hostData, size_t n) const {
        if (!valid() || n > count) return cudaErrorInvalidValue;
        return cudaMemcpy(hostData, ptr, n * sizeof(T), cudaMemcpyDeviceToHost);
    }
};

class CudaStream {
private:
    cudaStream_t stream;
    bool ownsStream;
    
public:
    CudaStream() : ownsStream(true) {
        cudaError_t err = cudaStreamCreate(&stream);
        if (err == cudaSuccess) {
            printf("  [RAII] Created stream %p\n", stream);
        } else {
            stream = nullptr;
            ownsStream = false;
        }
    }
    
    ~CudaStream() {
        if (ownsStream && stream) {
            printf("  [RAII] Destroying stream %p\n", stream);
            cudaStreamDestroy(stream);
        }
    }
    
    CudaStream(const CudaStream&) = delete;
    CudaStream& operator=(const CudaStream&) = delete;
    
    cudaStream_t get() { return stream; }
    bool valid() const { return stream != nullptr; }
    
    cudaError_t synchronize() { 
        return valid() ? cudaStreamSynchronize(stream) : cudaErrorInvalidValue; 
    }
};

__global__ void raiiTestKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = sqrtf((float)idx);
    }
}

void exercise2_raii_resource_manager() {
    printf("=== Exercise 2: RAII Resource Manager ===\n\n");
    
    printf("Creating RAII-managed resources in a scope...\n\n");
    
    {
        // All resources will be automatically cleaned up when leaving scope
        CudaBuffer<float> buffer(1024);
        CudaStream stream;
        
        if (buffer.valid() && stream.valid()) {
            printf("\nLaunching kernel with RAII resources...\n");
            raiiTestKernel<<<4, 256, 0, stream.get()>>>(buffer.get(), buffer.size());
            stream.synchronize();
            printf("Kernel completed!\n\n");
            
            // Copy result back
            float hostData[10];
            buffer.copyToHost(hostData, 10);
            printf("First 10 results: ");
            for (int i = 0; i < 10; i++) {
                printf("%.2f ", hostData[i]);
            }
            printf("\n\n");
        }
        
        printf("Leaving scope - resources will be automatically freed:\n");
    }
    
    printf("\nAll RAII resources cleaned up automatically!\n\n");
}

// =============================================================================
// Exercise 3: Failover Handler
// Design graceful GPU failure detection and recovery
// =============================================================================

class GPUFailoverManager {
private:
    int primaryDevice;
    int backupDevice;
    bool primaryFailed;
    int currentDevice;
    
    bool testDevice(int device) {
        cudaError_t err = cudaSetDevice(device);
        if (err != cudaSuccess) return false;
        
        // Try a small allocation to verify device is working
        void* testPtr;
        err = cudaMalloc(&testPtr, 1024);
        if (err != cudaSuccess) return false;
        
        cudaFree(testPtr);
        return true;
    }

public:
    GPUFailoverManager(int primary = 0, int backup = -1) 
        : primaryDevice(primary), backupDevice(backup), 
          primaryFailed(false), currentDevice(primary) {
        
        int deviceCount;
        cudaGetDeviceCount(&deviceCount);
        
        if (backupDevice < 0 && deviceCount > 1) {
            backupDevice = (primaryDevice + 1) % deviceCount;
        }
    }
    
    bool initialize() {
        printf("  Initializing failover manager...\n");
        printf("  Primary device: %d\n", primaryDevice);
        printf("  Backup device: %d\n", backupDevice);
        
        if (!testDevice(primaryDevice)) {
            printf("  WARNING: Primary device failed initial test!\n");
            return failoverToBackup();
        }
        
        printf("  Primary device operational\n");
        currentDevice = primaryDevice;
        return true;
    }
    
    bool failoverToBackup() {
        if (backupDevice < 0) {
            printf("  ERROR: No backup device configured!\n");
            return false;
        }
        
        printf("  Attempting failover to backup device %d...\n", backupDevice);
        
        if (!testDevice(backupDevice)) {
            printf("  ERROR: Backup device also failed!\n");
            return false;
        }
        
        primaryFailed = true;
        currentDevice = backupDevice;
        printf("  SUCCESS: Failover to device %d complete\n", backupDevice);
        return true;
    }
    
    bool executeWithFailover(void (*workFunc)(int)) {
        printf("\n  Executing work on device %d...\n", currentDevice);
        
        cudaError_t err = cudaSetDevice(currentDevice);
        if (err != cudaSuccess) {
            printf("  Device %d unavailable, attempting failover...\n", currentDevice);
            if (!failoverToBackup()) {
                return false;
            }
            cudaSetDevice(currentDevice);
        }
        
        workFunc(currentDevice);
        
        // Check for execution errors
        err = cudaDeviceSynchronize();
        if (err != cudaSuccess) {
            printf("  Execution error on device %d: %s\n", 
                   currentDevice, cudaGetErrorString(err));
            
            if (!primaryFailed && backupDevice >= 0) {
                printf("  Retrying on backup device...\n");
                if (failoverToBackup()) {
                    cudaSetDevice(currentDevice);
                    workFunc(currentDevice);
                    err = cudaDeviceSynchronize();
                    if (err == cudaSuccess) {
                        printf("  Retry succeeded on backup device\n");
                        return true;
                    }
                }
            }
            return false;
        }
        
        return true;
    }
    
    int getCurrentDevice() const { return currentDevice; }
    bool isPrimaryFailed() const { return primaryFailed; }
};

__global__ void failoverTestKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = (float)idx * 2.0f;
    }
}

void testWorkload(int device) {
    float* d_data;
    cudaMalloc(&d_data, 1024 * sizeof(float));
    failoverTestKernel<<<4, 256>>>(d_data, 1024);
    cudaDeviceSynchronize();
    cudaFree(d_data);
    printf("  Workload completed on device %d\n", device);
}

void exercise3_failover_handler() {
    printf("=== Exercise 3: Failover Handler ===\n\n");
    
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    printf("Available devices: %d\n\n", deviceCount);
    
    GPUFailoverManager failover(0, deviceCount > 1 ? 1 : -1);
    
    if (!failover.initialize()) {
        printf("Failed to initialize failover manager!\n");
        return;
    }
    
    printf("\nExecuting workload with failover protection:\n");
    bool success = failover.executeWithFailover(testWorkload);
    
    printf("\nExecution Result:\n");
    printf("  Success: %s\n", success ? "YES" : "NO");
    printf("  Final device: %d\n", failover.getCurrentDevice());
    printf("  Primary failed: %s\n\n", failover.isPrimaryFailed() ? "YES" : "NO");
}

int main() {
    printf("╔══════════════════════════════════════════════════════════════╗\n");
    printf("║   Week 18 Day 4: Production Patterns - CUDA C++ Exercises    ║\n");
    printf("╚══════════════════════════════════════════════════════════════╝\n\n");
    
    exercise1_health_monitor();
    exercise2_raii_resource_manager();
    exercise3_failover_handler();
    
    printf("All exercises completed!\n");
    return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_75 -o production_patterns_exercises production_patterns_exercises.cu && ./production_patterns_exercises

### 🔶 Python/Numba Exercises (Optional)

The following exercises use Python with PyCUDA/Numba for those who prefer Python-based GPU programming. Production patterns like health monitoring and failover are commonly implemented in Python orchestration layers.

**Python Exercise Ideas:**
1. Build a GPU health monitoring dashboard using `py3nvml` and Flask/Streamlit
2. Implement a Python resource manager with automatic cleanup using context managers
3. Create a production-ready GPU failover system with health checks and alerting

## 🎯 Key Takeaways

### Production Patterns Mastered! 🏨

You've learned to run GPU operations like a 5-star hotel chain:

| Pattern | Description | Hotel Analogy |
|---------|-------------|---------------|
| **Health Monitoring** | Temperature, memory, ECC | Room sensors dashboard |
| **Failover Systems** | Handle GPU failures gracefully | Backup power/rooms |
| **RAII Wrappers** | Automatic resource cleanup | Housekeeping protocols |
| **Comprehensive Logging** | Errors, warnings, metrics | Guest registry & incidents |
| **Error Path Testing** | Verify recovery procedures | Fire drills & training |

### 🏨 The Hotel Management Principle
> "A production GPU system needs the same care as a 5-star hotel: constant health monitoring, graceful failover, automatic cleanup, and staff ready for any emergency."

---

## 🎓 Week 18 Complete: Enterprise GPU Management

### 🏆 Your MIG & Production Journey

```
Week 18 Achievement Unlocked! 🏅
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Day 1: MIG Fundamentals     🏨 Hotel partitioning
Day 2: MIG Configuration    🔧 Creating private suites  
Day 3: Error Management     🚨 Emergency protocols
Day 4: Production Patterns  ⭐ 5-star operations
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        ENTERPRISE GPU MASTER!
```

### 🎊 Congratulations on Completing the CUDA Curriculum!

You've completed an incredible 18-week journey from GPU basics to enterprise-level GPU management:

| Phase | Weeks | Key Skills |
|-------|-------|------------|
| **Foundations** | 1-4 | Threads, memory, optimization basics |
| **Optimization** | 5-8 | Profiling, occupancy, performance tuning |
| **Advanced** | 9-12 | Streams, graphs, cooperative groups, multi-GPU |
| **Special Topics** | 13-16 | Libraries, deep learning, graphics interop |
| **Enterprise** | 17-18 | Debugging, profiling, MIG, production |

### 🚀 What's Next?
- Deploy MIG-aware applications in production
- Contribute to GPU-accelerated open source projects
- Explore emerging NVIDIA technologies (Grace Hopper, Blackwell)
- Share your knowledge with the community!

**Thank you for learning with us! 🙏**